In [28]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [29]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [30]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [31]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [32]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [33]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [34]:
X['Date'] = X.index

In [35]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [36]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [37]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [38]:
len(y_sample), len(X_sample)

(1252, 1252)

In [39]:
torch.set_default_dtype(torch.float64)

In [40]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [ ]:
test_nan_indexes = []
for tup in range(len(test_data)):
    if (torch.any(test_data[tup][0].isnan(), dim=0)):
        print(f"Found an tensor with nan at index {tup}")
        test_nan_indexes.append(tup)

Found an tensor with nan at index 270
Found an tensor with nan at index 271
Found an tensor with nan at index 272


In [62]:
for i in test_nan_indexes:
    temp_tensor = test_data[i][0]
    temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
    temp_tuple = (temp_tensor, test_data[i][1])
    test_data[i] = temp_tuple

In [43]:
temp_tensor = train_data[0][0]
temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
temp_tuple_1 = (temp_tensor, train_data[0][1])


temp_tensor = train_data[1][0]
temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
temp_tuple_2 = (temp_tensor, train_data[1][1])

In [44]:
train_data[0] = temp_tuple_1
train_data[1] = temp_tuple_2

In [45]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32,)
test_dataloader= DataLoader(test_data, batch_size=32,)

In [46]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        print(X)
        print(torch.nan_to_num(X, nan=0))
        count+=1
    batch +=1


In [47]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, 15),
            nn.ReLU()
        )
        self.l3 = nn.Linear(15, output_features)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [48]:
mlpmodel = MLP(8, 30, 1)

In [49]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=0.01)

In [50]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 3.0065523646714762
We're on batch: 2 + Loss: 2.105431839632205
We're on batch: 3 + Loss: 1.7770167432275847
We're on batch: 4 + Loss: 1.6216967967731897
We're on batch: 5 + Loss: 1.4392611316142916
We're on batch: 6 + Loss: 1.2765030382593323
We're on batch: 7 + Loss: 1.1307601626132135
We're on batch: 8 + Loss: 1.0118672589552455
We're on batch: 9 + Loss: 0.9122894425235294
We're on batch: 10 + Loss: 0.8313960827147454
We're on batch: 11 + Loss: 0.761140997209798
We're on batch: 12 + Loss: 0.6991363000115638
We're on batch: 13 + Loss: 0.645619378015401
We're on batch: 14 + Loss: 0.5997142284212141
We're on batch: 15 + Loss: 0.5598476168104327
We're on batch: 16 + Loss: 0.5248901761399245
We're on batch: 17 + Loss: 0.4940476218353875
We're on batch: 18 + Loss: 0.4667614609009936
We're on batch: 19 + Loss: 0.44262298885771223
We're on batch: 20 + Loss: 0.420715622781466
We're on batch: 21 + Loss: 0.4006956493393952
We're on batch: 

In [53]:
with torch.inference_mode():
    mlpmodel.eval()
    for batch, (X, y) in enumerate(test_dataloader):
        preds = mlpmodel(X)
        loss = loss_fn(torch.unsqueeze(y, dim=1), preds)
        print(f"On Batch: {batch} + With Loss: {loss/batch}")

On Batch: 0 + With Loss: inf
On Batch: 1 + With Loss: 0.0010275667426416883
On Batch: 2 + With Loss: 0.0006296117461702505
On Batch: 3 + With Loss: 0.001287243992785148
On Batch: 4 + With Loss: 0.007956507984416866
On Batch: 5 + With Loss: 0.00564706654680707
On Batch: 6 + With Loss: 0.000552770069253797
On Batch: 7 + With Loss: 0.0010698695987964356
On Batch: 8 + With Loss: nan
